In [37]:
import pandas as pd
import numpy as np
#from pivottablejs import pivot_ui
import pyodbc
from geopy import Point
from geopy.distance import distance
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [49]:
cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
            "Server=DWHDB;"
            "Database=DWBIBFI2_ODS;"
            "UID=Data_Inovasion;"
            "PWD=Inovasion999;")
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()

In [50]:
query = """
with cust_contract_active as ( --538.855
select distinct b.SK_Customer
from DWBIBFI2_DWH.dbo.Fact_Contract_SS a (nolock)
left join DWBIBFI2_DWH.dbo.Fact_Contract b (nolock) on a.ID_Application = b.ID_Application
where 	SK_Time = 20230601 and SK_Contract_Active = 2
),
cust_address AS (
SELECT cif_id, address_type, address, address_detail, district, subdistrict, city, province, zip_code, lat, lng as long 
FROM DWBIBFI2_STG.dbo.STG_BravoLOS_customer_cif_address (nolock)
where address_type = 'RESIDENCE'
),
cust_id AS (
SELECT cif_id, confins_customer_id
FROM DWBIBFI2_STG.dbo.STG_BravoLOS_customer_cif (nolock)
),
address_bravo as (
SELECT DISTINCT confins_customer_id, address_type, address, address_detail, district, subdistrict, city, province, zip_code, lat, long
FROM cust_address 
LEFT JOIN cust_id ON cust_address.cif_id= cust_id.cif_id
WHERE confins_customer_id IS NOT NULL
)
select 
		ID_Customer, 
		upper(dc.ResidenceCity) 		as confin_city,			upper(br.city)			as bravo_city,
		upper(dc.ResidenceKecamatan) 	as confin_kecamatan, 	upper(br.district)		as bravo_kecamatan,
		upper(dc.ResidenceKelurahan)	as confin_kelurahan,	upper(br.subdistrict)	as bravo_kelurahan,
		dc.ZipCode 						as confin_zipcode,		br.zip_code 			as bravo_zipcode,
		upper(pc.ResidenceAddress) 		as confin_address,  	upper(br.address)		as bravo_address, 
		cast(dc.Longitude as float)		as confin_Longitude,	br.long					as bravo_Longitude,
		cast(dc.Latitude as float)		as confin_Latitude,		br.lat					as bravo_Latitude,
		case when (cast(dc.ZipCode as int) = cast(br.zip_code as int)) then 100 else 0 end as score_ZipCode,
		case when (cast(dc.Longitude as float) = 0 or cast(dc.Latitude as float) = 0) then 'Blank' end as Long_lat_Confins, 
		case when (br.long = 0 or br.lat = 0) then 'Blank' end as Long_lat_Bravo
from cust_contract_active ca
Left Join DWBIBFI2_DWH.dbo.Dim_CUSTOMER dc (nolock) On ca.SK_Customer = dc.SK_Customer
Left join DWBIBFI2_STG.dbo.STG_PersonalCustomer pc (nolock) On dc.ID_Customer = pc.CustomerID
inner join address_bravo br (nolock) On dc.ID_Customer = br.confins_customer_id --7155
"""
#cursor.execute(query)

In [51]:
data = pd.read_sql(query, cnxn)

C:\Users\099692\AppData\Local\Temp\ipykernel_1756\3477599559.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, cnxn)


In [54]:
data.to_excel("intersect.xlsx", index=False)

In [18]:
result = (data.groupby(['Long_lat_Bravo']).size()
   .sort_values(ascending=False) 
   .reset_index(name='count') )
   #.drop_duplicates(subset='len'))
print(result)

  Long_lat_Bravo  count
0          Blank   6617


In [55]:
data['point_confin'] = data.apply(lambda row: Point(latitude=row['confin_Latitude'], longitude=row['confin_Longitude']), axis=1)
data['bravo_Latitude'] = data.bravo_Latitude.replace(0, None)
data['bravo_Longitude'] = data.bravo_Longitude.replace(0, None)
data['point_bravo'] = data.apply(lambda row: Point(latitude=row['bravo_Latitude'], longitude=row['bravo_Longitude']), axis=1)

In [56]:
data['distance_km'] = data.apply(lambda row: distance(row['point_confin'], row['point_bravo']).km if row['Long_lat_Bravo'] != 'Blank' else float('nan'), axis=1)

In [57]:
data['distance_km_group'] = np.select(
    [
        data['distance_km'].between(10, 999999999999, inclusive=True),
        data['distance_km'].between(5, 10, inclusive=True),
        data['distance_km'].between(2.5, 5, inclusive=True),
        data['distance_km'].between(1, 2.5, inclusive=True),
        data['distance_km'].between(0.5, 1, inclusive=True),
        data['distance_km'].between(0.25, 0.5, inclusive=True),
        data['distance_km'].between(0.1, 0.25, inclusive=True),
        data['distance_km'].between(0, 0.1, inclusive=True)
    ], 
    [ 
        'h) > 10 Km',
        'g) 5 - 10 Km', 
        'f) 2.5 - 5 Km',
        'e) 1 - 2.5 Km', 
        'd) 500 m - 1 Km',
        'c) 250 - 500 m', 
        'b) 100 - 250 m', 
        'a) 0 - 100 m'
    ], 
    default='Unknown'
)

C:\Users\099692\AppData\Local\Temp\ipykernel_1756\2066469328.py:3: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  data['distance_km'].between(10, 999999999999, inclusive=True),
C:\Users\099692\AppData\Local\Temp\ipykernel_1756\2066469328.py:4: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  data['distance_km'].between(5, 10, inclusive=True),
C:\Users\099692\AppData\Local\Temp\ipykernel_1756\2066469328.py:5: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  data['distance_km'].between(2.5, 5, inclusive=True),
C:\Users\099692\AppData\Local\Temp\ipykernel_1756\2066469328.py:6: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  data['distance_km'].between(1, 2.5, inclusive=True),
C:\Users\099692\AppData\Local\Temp\ipykernel_1756\2066469328.py:7: FutureWa

In [58]:
result = (data.groupby(['distance_km_group']).size()
   .sort_values(ascending=False) 
   .reset_index(name='count') )
   #.drop_duplicates(subset='len'))
print(result)

  distance_km_group  count
0           Unknown   6617
1     e) 1 - 2.5 Km    145
2        h) > 10 Km    118
3   d) 500 m - 1 Km     97
4    c) 250 - 500 m     45
5     f) 2.5 - 5 Km     42
6    b) 100 - 250 m     39
7      g) 5 - 10 Km     27
8      a) 0 - 100 m     25


In [65]:
data.head()

,ID_Customer,confin_city,bravo_city,confin_kecamatan,bravo_kecamatan,confin_kelurahan,bravo_kelurahan,confin_zipcode,bravo_zipcode,confin_address,...,point_confin,point_bravo,distance_km,distance_km_group,score_city,score_kecamatan,score_kelurahan,score_address,avg_address_similarity,avg_address_similarity_group
0,2011114880003845,GRESIK,GRESIK,KEBOMAS,KEBOMAS,SIDOMUKTI,SIDOMUKTI,61124,61124,SUNAN GIRI 13.R,...,"7 10m 51.6s S, 112 37m 26.4s E","0 0m 0s N, 0 0m 0s E",NaN,Unknown,100,100,100,100,100.0,a) 90 - 100
1,2012094900003917,BOJONEGORO,BOJONEGORO,KEPOHBARU,KEPOHBARU,SUMBERGEDE,SUMBERAGUNG,62194,62194,DUSUN SUMBERAGUNG,...,"7 9m 3.6s S, 112 6m 50.4s E","0 0m 0s N, 0 0m 0s E",NaN,Unknown,100,100,67,100,93.4,a) 90 - 100
2,2012104880006190,GRESIK,GRESIK,UJUNGPANGKAH,UJUNGPANGKAH (UJUNG PANGKAH),SEKAPUK,SEKAPUK,61154,61154,SEKAPUK RT/RW 002/004,...,"7 14m 2.4s S, 112 45m 21.6s E","0 0m 0s N, 0 0m 0s E",NaN,Unknown,100,100,100,100,100.0,a) 90 - 100
3,2012114900004048,BOJONEGORO,BOJONEGORO,KANOR,KANOR,SROYO,SROYO,62193,62193,SROYO,...,"7 9m 25.2s S, 112 2m 34.8s E","0 0m 0s N, 0 0m 0s E",NaN,Unknown,100,100,100,100,100.0,a) 90 - 100
4,2012124710005383,MOJOKERTO,KAB. MOJOKERTO,KEMLAGI,KEMLAGI,MOJOSARIREJO,MOJOSARIREJO,61353,61353,DUSUN JRAMBE,...,"7 25m 51.6s S, 112 20m 52.8s E","0 0m 0s N, 0 0m 0s E",NaN,Unknown,100,100,100,100,100.0,a) 90 - 100


In [60]:
data['score_city'] = data.apply(lambda x: fuzz.token_set_ratio(x['confin_city'], x['bravo_city']), axis=1)
data['score_kecamatan'] = data.apply(lambda x: fuzz.token_set_ratio(x['confin_kecamatan'], x['bravo_kecamatan']), axis=1)
data['score_kelurahan'] = data.apply(lambda x: fuzz.token_set_ratio(x['confin_kelurahan'], x['bravo_kelurahan']), axis=1)
data['score_address'] = data.apply(lambda x: fuzz.token_set_ratio(x['confin_address'], x['bravo_address']), axis=1)

In [62]:
data['avg_address_similarity'] = data[['score_ZipCode', 'score_city','score_kecamatan','score_kelurahan','score_address']].mean(axis=1)

In [64]:
data['avg_address_similarity_group'] = np.select(
    [
        data['avg_address_similarity'].between(90, 100, inclusive=True),
        data['avg_address_similarity'].between(80, 90, inclusive=True),
        data['avg_address_similarity'].between(70, 80, inclusive=True),
        data['avg_address_similarity'].between(60, 70, inclusive=True),
        data['avg_address_similarity'].between(50, 60, inclusive=True),
        data['avg_address_similarity'].between(40, 50, inclusive=True),
        data['avg_address_similarity'].between(30, 40, inclusive=True),
        data['avg_address_similarity'].between(20, 30, inclusive=True),
        data['avg_address_similarity'].between(10, 20, inclusive=True),
        data['avg_address_similarity'].between(0, 10, inclusive=True)
    ], 
    [
        'a) 90 - 100', 
        'b) 80 - 90', 
        'c) 70 - 80',
        'd) 60 - 70', 
        'e) 50 - 60',
        'f) 40 - 50', 
        'g) 30 - 40',
        'h) 20 - 30', 
        'i) 10 - 20', 
        'j) 0 - 10'
    ], 
    default='Unknown'
)

C:\Users\099692\AppData\Local\Temp\ipykernel_1756\50527795.py:3: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  data['avg_address_similarity'].between(90, 100, inclusive=True),
C:\Users\099692\AppData\Local\Temp\ipykernel_1756\50527795.py:4: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  data['avg_address_similarity'].between(80, 90, inclusive=True),
C:\Users\099692\AppData\Local\Temp\ipykernel_1756\50527795.py:5: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  data['avg_address_similarity'].between(70, 80, inclusive=True),
C:\Users\099692\AppData\Local\Temp\ipykernel_1756\50527795.py:6: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  data['avg_address_similarity'].between(60, 70, inclusive=True),
C:\Users\099692\AppData\Local\Temp\ipykernel_17

In [66]:
data.to_excel("intersect_olah.xlsx", index=False)